In [1]:
!pip install arcticdb
!pip install pandas
!pip install pyexiftool
# !apt install libimage-exiftool-perl
from arcticdb import Arctic, QueryBuilder
import exiftool
import sys
import os
import pandas as pd 
import numpy as np
import requests
import typing
# import subprocess
# from arcticdb_ext.exceptions import InternalExecption

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.54.0
    Uninstalling grpcio-1.54.0:
      Successfully uninstalled grpcio-1.54.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels

In [2]:
def _handle_errors(msg):
  print(Exception(msg))

In [3]:
ac = Arctic(f"lmdb:///{os.getcwd()}/new_folder")
name_db = "testing.arctic"
try:
  ac.get_library(name_db)
  print("Found lib")
except:
  print("Not found")
try:
  ac.create_library(name_db)
  print("Created lib")
except:
  print("Lib exists")


ac.list_libraries()

Not found
Created lib


['testing.arctic']

# Populating dataframe with real and public data



In [4]:
lib = ac[name_db]
try:
  lib.read("my_data")
  print("Data already existing, skipping")
except:
  try:
    df = pd.read_csv("https://dados.cvm.gov.br/dados/FI/DOC/EVENTUAL/DADOS/eventual_fi_2023.csv",sep=";",encoding = "unicode_escape")
    print("Created the dataframe with cvm data")
  except Exception as er:
    print(er)
    print("An error occurred when trying to generate the dataframe with the cvm data, modify or let a standard dataframe be generated as shown in the docs")
    NUM_COLUMNS=10
    NUM_ROWS=100_000
    df = pd.DataFrame(np.random.randint(0,100,size=(NUM_ROWS, NUM_COLUMNS)), columns=[f"COL_{i}" for i in range(NUM_COLUMNS)], index=pd.date_range('2000', periods=NUM_ROWS, freq='h'))
  lib.write("my_data", df)
# print(lib)
# data = lib.read("my_data",as_of=0)
# print(lib.tail("my_data",  5, as_of=0).data)

# data = lib.read("my_data")
# print(data)
# print(data.data)

Created the dataframe with cvm data


# Create query builder using arcticdb





In [5]:
def _download_files(link: str = "") -> typing.Union[bytes,None]:
    try:
        response = requests.get(link)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(f"Ocorreu um erro ao baixar o arquivo {link}: {e}")
        return None

In [6]:
query = QueryBuilder()
query = query[query["TP_DOC"] == "REGUL FDO"]
 
data = lib.read("my_data", query_builder=query).data[["CNPJ_FUNDO", "DT_RECEB", "TP_FUNDO", "LINK_ARQ"]]

for _index, _row in data.iterrows():
  print(_row["LINK_ARQ"])
  arq = _download_files(_row["LINK_ARQ"])
  #TODO Adjusting this code
  # with exiftool.ExifTool() as et:
  #   # Analisar as informações do arquivo diretamente do conteúdo
  #   metadata = et.get_metadata(arq.content)
  #   print(metadata)
  break

https://web.cvm.gov.br/app/fundosweb/regulamentoFundo/por/arquivo/DOC_REGUL_83365_82299.pdf
